In [5]:
import os
import json
import datetime as dt
import pandas as pd
import numpy as np
import hopsworks

# =========================
# CONFIG (CAMBIA QUI)
# =========================
FV_NAME = os.getenv("FV_NAME", "pollen_stockholm_fv")   # <-- nome Feature View
FV_VERSION = int(os.getenv("FV_VERSION", "1"))

MODEL_NAME = os.getenv("MODEL_NAME", "pollen_stockholm_xgboost")  # <-- nome nel Model Registry
MODEL_VERSION = os.getenv("MODEL_VERSION")  # None => prendi latest

CITY = os.getenv("CITY", "stockholm")

FORECAST_HORIZON_DAYS = int(os.getenv("FORECAST_HORIZON_DAYS", "7"))

# Dove salvare output per UI dentro Hopsworks (Dataset)
DATASET_BASE_DIR = os.getenv("DATASET_BASE_DIR", "Resources/pollen_forecasts")

# Se vuoi anche un FG di monitoring (opzionale)
WRITE_MONITORING_FG = os.getenv("WRITE_MONITORING_FG", "true").lower() == "true"
MONITORING_FG_NAME = os.getenv("MONITORING_FG_NAME", "pollen_predictions")
MONITORING_FG_VERSION = int(os.getenv("MONITORING_FG_VERSION", "1"))

# Colonne che NON devono entrare nel modello (come nel training)
FEATURES_TO_DROP = ["city", "country", "date", "date_str", "time", "unix_time"]

# =========================
# UTILS
# =========================
def _safe_to_numeric(df: pd.DataFrame) -> pd.DataFrame:
    """Converte tutto a numerico per XGBoost, coerente col fix del training."""
    out = df.apply(pd.to_numeric, errors="coerce").fillna(0.0).astype("float32")
    return out

def _infer_target_columns(pred_array: np.ndarray, fv) -> list[str]:
    """
    Prova a ricostruire i nomi target in modo robusto.
    - Se sai già i target: mettili hard-coded.
    - Altrimenti prova da feature view/metadata.
    """
    # Se vuoi hardcodare:
    # return ["alder_pollen", "birch_pollen", "grass_pollen", ...]
    # Fallback generico:
    return [f"target_{i}" for i in range(pred_array.shape[1])]

# =========================
# MAIN
# =========================
print("🔌 Login Hopsworks...")
project = hopsworks.login()
fs = project.get_feature_store()
mr = project.get_model_registry()
dataset_api = project.get_dataset_api()

print(f"📌 Feature View: {FV_NAME} v{FV_VERSION}")
feature_view = fs.get_feature_view(name=FV_NAME, version=FV_VERSION)
print(f"📦 Recupero modello: {MODEL_NAME} (versione: {MODEL_VERSION or 'latest'})")
if MODEL_VERSION:
    hs_model = mr.get_model(MODEL_NAME, version=int(MODEL_VERSION))
else:
    hs_model = mr.get_model(MODEL_NAME)

model_dir = hs_model.download()
print(f"✅ Modello scaricato in: {model_dir}")

# --- Debug: stampa contenuto cartelle (opzionale ma utile) ---
print("📦 Contenuto model_dir:")
for root, dirs, files in os.walk(model_dir):
    rel = os.path.relpath(root, model_dir)
    print(f"\nDIR: {rel}")
    for f in files:
        print("  -", f)

# =========================================================
# Loader robusto (coerente col tuo training)
# 1) prova prima pollen_xgboost_model.pkl
# 2) fallback: cerca qualsiasi .pkl/.joblib/.pickle
# =========================================================
import joblib
import pickle

def find_file_recursive(base_dir: str, filename: str) -> str | None:
    for root, _, files in os.walk(base_dir):
        if filename in files:
            return os.path.join(root, filename)
    return None

loaded_model = None

# 1) Nome atteso dal tuo notebook 3
preferred = find_file_recursive(model_dir, "pollen_xgboost_model.pkl")
if preferred is not None:
    loaded_model = joblib.load(preferred)
    print(f"✅ Modello caricato da (preferred): {preferred}")
else:
    # 2) fallback: qualsiasi file modello
    candidates = []
    for root, _, files in os.walk(model_dir):
        for f in files:
            if f.lower().endswith((".joblib", ".pkl", ".pickle")):
                candidates.append(os.path.join(root, f))

    print("🔎 Candidati trovati:", candidates)

    # prova joblib prima
    for path in candidates:
        try:
            loaded_model = joblib.load(path)
            print(f"✅ Modello caricato con joblib da: {path}")
            break
        except Exception:
            pass

    # poi pickle
    if loaded_model is None:
        for path in candidates:
            try:
                with open(path, "rb") as f:
                    loaded_model = pickle.load(f)
                print(f"✅ Modello caricato con pickle da: {path}")
                break
            except Exception:
                pass

if loaded_model is None:
    raise FileNotFoundError(
        f"❌ Non ho trovato/letto un artifact modello dentro: {model_dir}\n"
        "Attesi: pollen_xgboost_model.pkl (dal training) oppure un qualsiasi .pkl/.joblib/.pickle."
    )

print("✅ Modello caricato.")



🔌 Login Hopsworks...
2025-12-20 18:40:51,829 INFO: Closing external client and cleaning up certificates.
2025-12-20 18:40:51,832 INFO: Connection closed.
2025-12-20 18:40:51,834 INFO: Initializing external client
2025-12-20 18:40:51,834 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-12-20 18:40:52,595 WARNING: UserWarning: The installed hopsworks client version 4.6.0 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-12-20 18:40:53,495 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1267872
📌 Feature View: pollen_stockholm_fv v1
📦 Recupero modello: pollen_stockholm_xgboost (versione: latest)
2025-12-20 18:40:56,906 WARNING: VersionWarning: No version provided for getting model `pollen_stockholm_xgboost`, defaulting to `1`.



Downloading: 0.000%|          | 0/10825223 elapsed<00:00 remaining<?

✅ Modello scaricato in: /var/folders/57/stqc67wn4wd3zdl6qg664yh40000gn/T/7c587be1-c869-4cd9-b6cd-a108d56bbc71/pollen_stockholm_xgboost/1
📦 Contenuto model_dir:

DIR: .
  - pollen_xgboost_model.pkl
✅ Modello caricato da (preferred): /var/folders/57/stqc67wn4wd3zdl6qg664yh40000gn/T/7c587be1-c869-4cd9-b6cd-a108d56bbc71/pollen_stockholm_xgboost/1/pollen_xgboost_model.pkl
✅ Modello caricato.


In [10]:
import pandas as pd
import datetime as dt

today = dt.date.today()
start_date = today + dt.timedelta(days=1)
end_date = start_date + dt.timedelta(days=FORECAST_HORIZON_DAYS - 1)

print(f"📅 Forecast window: {start_date} -> {end_date} ({FORECAST_HORIZON_DAYS} giorni)")

# =========================================================
# FIX DEFINITIVO: usa i dati materializzati del training dataset
# (evita Feature Query Service che non supporta delta connector)
# =========================================================

# Se vuoi forzare una versione specifica del training dataset, setta TRAIN_DATASET_VERSION
TRAIN_DATASET_VERSION = int(os.getenv("TRAIN_DATASET_VERSION", "1"))

print(f"📦 Carico train/test split version={TRAIN_DATASET_VERSION} (materializzato) ...")
X_train, X_test, y_train_tmp, y_test_tmp = feature_view.get_train_test_split(TRAIN_DATASET_VERSION)

# Usiamo X_test come "proxy" del futuro: prendiamo le righe più recenti
if "date" not in X_test.columns:
    raise ValueError(f"❌ X_test non contiene 'date'. Colonne: {list(X_test.columns)}")

X_test = X_test.copy()
X_test["date"] = pd.to_datetime(X_test["date"])

# Ordina per data e prendi gli ultimi HORIZON_DAYS
X_test = X_test.sort_values("date").reset_index(drop=True)

# Se vuoi proprio la finestra start/end, prova a filtrare; se vuoto, fallback agli ultimi 7
mask = (X_test["date"].dt.date >= start_date) & (X_test["date"].dt.date <= end_date)
batch_df = X_test.loc[mask].copy()

if batch_df.empty:
    print("⚠️ Nessuna riga esatta nel range futuro. Fallback: uso gli ultimi 7 giorni disponibili in X_test.")
    batch_df = X_test.tail(FORECAST_HORIZON_DAYS).copy()

# Se city non c'è, aggiungila
if "city" not in batch_df.columns:
    batch_df["city"] = CITY

print(f"✅ Batch data pronti: {len(batch_df)} righe, {batch_df.shape[1]} colonne")
print(batch_df[["date"]].head())

# =========================================================
# 2) Selezione feature e FIX dtype (come training)
# =========================================================
train_cols = [c for c in batch_df.columns if c not in FEATURES_TO_DROP]
X_batch_raw = batch_df[train_cols].copy()

X_batch = _safe_to_numeric(X_batch_raw)


📅 Forecast window: 2025-12-21 -> 2025-12-27 (7 giorni)
📦 Carico train/test split version=1 (materializzato) ...
⚠️ Nessuna riga esatta nel range futuro. Fallback: uso gli ultimi 7 giorni disponibili in X_test.
✅ Batch data pronti: 7 righe, 25 colonne
                          date
1562 2025-12-05 00:00:00+00:00
1563 2025-12-06 00:00:00+00:00
1564 2025-12-09 00:00:00+00:00
1565 2025-12-13 00:00:00+00:00
1566 2025-12-15 00:00:00+00:00


In [12]:
# =========================
# 3) Predizione
# =========================
print("🤖 Predict...")

POLLENS = ["alder_pollen", "birch_pollen", "grass_pollen", "mugwort_pollen"]

y_pred = loaded_model.predict(X_batch)

# Normalizza shape
y_pred = np.asarray(y_pred)
if y_pred.ndim == 1:
    y_pred = y_pred.reshape(-1, 1)

# Controllo dimensioni: devono essere 4 output
if y_pred.shape[1] != len(POLLENS):
    raise ValueError(
        f"❌ Il modello ha prodotto {y_pred.shape[1]} output, ma la UI si aspetta {len(POLLENS)}: {POLLENS}.\n"
        "Probabile mismatch tra ordine/numero target del training e quello atteso dalla UI."
    )

pred_df = pd.DataFrame(y_pred, columns=POLLENS)

# Costruisci out_df con date/date_str/city + predizioni
out_df = batch_df.copy()

# Assicura colonne tempo
out_df["date"] = pd.to_datetime(out_df["date"])
if "date_str" not in out_df.columns:
    out_df["date_str"] = out_df["date"].dt.strftime("%Y-%m-%d")

# Assicura city
if "city" not in out_df.columns:
    out_df["city"] = CITY

# Tieni solo le colonne utili + pred
out_df = pd.concat(
    [out_df[["date", "date_str", "city"]].reset_index(drop=True),
     pred_df.reset_index(drop=True)],
    axis=1
)

out_df = out_df.sort_values("date").reset_index(drop=True)

print("✅ Predizioni generate.")
print(out_df.head(3))


🤖 Predict...
✅ Predizioni generate.
                       date    date_str       city  alder_pollen  \
0 2025-12-05 00:00:00+00:00  2025-12-05  stockholm      0.001625   
1 2025-12-06 00:00:00+00:00  2025-12-06  stockholm     -0.000348   
2 2025-12-09 00:00:00+00:00  2025-12-09  stockholm      0.000079   

   birch_pollen  grass_pollen  mugwort_pollen  
0      0.000201      0.000016        0.000008  
1      0.000121     -0.000049       -0.000025  
2      0.000174      0.000074       -0.000005  


In [13]:
# =========================
# 4) Salvataggio output per UI (latest_forecasts.json)
# =========================
from datetime import datetime
import json

POLLENS = ["alder_pollen", "birch_pollen", "grass_pollen", "mugwort_pollen"]
FORECAST_PATH = "latest_forecasts.json"   # <-- deve matchare app.py

# out_df deve avere: date (datetime), e le 4 colonne POLLENS
ui_df = out_df.sort_values("date").head(FORECAST_HORIZON_DAYS).copy()
ui_df["date"] = pd.to_datetime(ui_df["date"]).dt.strftime("%Y-%m-%d")

payload = {
    "generated_at": datetime.now().strftime("%Y-%m-%d %H:%M"),
    "forecasts": {}
}

for p in POLLENS:
    if p not in ui_df.columns:
        raise ValueError(f"❌ Manca colonna '{p}' in out_df. Colonne: {list(ui_df.columns)}")

    payload["forecasts"][p] = [
        {"date": d, "value": float(v)}
        for d, v in zip(ui_df["date"].tolist(), ui_df[p].tolist())
    ]

with open(FORECAST_PATH, "w", encoding="utf-8") as f:
    json.dump(payload, f, ensure_ascii=False, indent=2)

print(f"✅ Scritto file UI: {FORECAST_PATH}")


✅ Scritto file UI: latest_forecasts.json


In [19]:
# =========================
# 5) (Opzionale) Monitoring Feature Group
# =========================


if WRITE_MONITORING_FG:
    print(f"📈 Writing monitoring FG: {MONITORING_FG_NAME} v{MONITORING_FG_VERSION}")

    POLLENS = ["alder_pollen", "birch_pollen", "grass_pollen", "mugwort_pollen"]

    # -------------------------------------------------
    # 1) Prepara dataframe pulito per monitoring
    # -------------------------------------------------
    mon_df = out_df.copy()
    mon_df["date"] = pd.to_datetime(mon_df["date"])

    if "date_str" not in mon_df.columns:
        mon_df["date_str"] = mon_df["date"].dt.strftime("%Y-%m-%d")

    if "city" not in mon_df.columns:
        mon_df["city"] = CITY

    keep_cols = ["city", "date", "date_str"] + POLLENS
    missing = [c for c in keep_cols if c not in mon_df.columns]
    if missing:
        raise ValueError(f"❌ Colonne mancanti per monitoring FG: {missing}")

    mon_df = mon_df[keep_cols].copy()

    # Forza pollini a float
    for p in POLLENS:
        mon_df[p] = (
            pd.to_numeric(mon_df[p], errors="coerce")
            .fillna(0.0)
            .astype("float32")
        )

    # -------------------------------------------------
    # 2) Deduplica su chiave primaria (city, date_str)
    # -------------------------------------------------
    mon_df = (
        mon_df
        .sort_values("date")                      # ordine deterministico
        .drop_duplicates(
            subset=["city", "date_str"],
            keep="last"                           # tieni previsione più recente
        )
        .reset_index(drop=True)
    )

    print(f"🔎 Monitoring DF dopo deduplica: {len(mon_df)} righe")

    # -------------------------------------------------
    # 3) Crea o recupera Feature Group
    # -------------------------------------------------
    mon_fg = fs.get_or_create_feature_group(
        name=MONITORING_FG_NAME,
        version=MONITORING_FG_VERSION,
        description="Pollen predictions for monitoring and hindcast",
        primary_key=["city", "date_str"],
        event_time="date"
    )

    # -------------------------------------------------
    # 4) Inserimento (insert semplice, robusto)
    # -------------------------------------------------
    mon_fg.insert(mon_df, wait=True)
    print("✅ Monitoring FG aggiornato.")
